Data Request:
Updated emissions inventory (CO2e and VMT) for King County, including cities 

Year 2023

In [1]:
import os
import pandas as pd

# Display results with comma and no decimal places
pd.options.display.float_format = '{:,.01f}'.format

veh_type_list = ['light', 'medium', 'heavy', 'transit']

# County Level VMT and Emissions

### CO2e

In [2]:
county = 'King'
output_dir = 'Y:/Air Quality/King County Emissions Inventory/2023/data/output'

results_df = pd.DataFrame()
df_running = pd.read_csv(os.path.join(output_dir, f'interpolated/{county}/2023/running_summary.csv'))
df_start = pd.read_csv(os.path.join(output_dir, f'interpolated/{county}/2023/start_summary.csv'))
for pollutant in ['CO2 Equivalent','Methane', 'N20']:
    # Running
    df_pollutant_running = df_running[df_running['pollutant_name']==pollutant].groupby('veh_type').sum()[['running_daily_tons']]
    df_pollutant_start = df_start[df_start['pollutant_name']==pollutant].groupby('veh_type').sum()[['start_tons']]
    df_pollutant = df_pollutant_running.merge(df_pollutant_start, on='veh_type')
    df_pollutant = df_pollutant.loc[veh_type_list]
    df_pollutant['pollutant_name'] = pollutant

    results_df = pd.concat([results_df, df_pollutant])


results_df['total_daily_tons'] = results_df['running_daily_tons'] + results_df['start_tons']

results_df = results_df[['pollutant_name','start_tons','running_daily_tons','total_daily_tons']]

# display with conditional formatting: no decimals for values > 10, two decimals otherwise
def fmt(x):
    if pd.isna(x):
        return ""
    try:
        if abs(x) > 10:
            return f"{x:,.0f}"
        else:
            return f"{x:,.2f}"
    except Exception:
        return x

county_emissions_results = results_df.copy()
results_df.style.format(fmt)


,pollutant_name,start_tons,running_daily_tons,total_daily_tons
veh_type,,,,
light,CO2 Equivalent,"1,096","15,655","16,751"
medium,CO2 Equivalent,65,"1,431","1,496"
heavy,CO2 Equivalent,1.39,"2,720","2,722"
transit,CO2 Equivalent,0.59,249,250
light,Methane,0.74,0.47,1.20
medium,Methane,0.17,0.25,0.42
heavy,Methane,0.00,0.15,0.16
transit,Methane,0.00,0.31,0.31
light,N20,0.26,0.18,0.44


In [3]:
# Total by emissions
results_df.groupby('pollutant_name').sum().style.format(fmt)

,start_tons,running_daily_tons,total_daily_tons
pollutant_name,,,
CO2 Equivalent,"1,163","20,056","21,219"
Methane,0.91,1.18,2.09
N20,0.30,0.56,0.85


In [4]:
results_df.to_csv(r"Y:\Air Quality\King County Emissions Inventory\2023\king_county_emissions.csv")

### VMT

In [5]:
df_vmt = pd.read_csv(os.path.join(output_dir, f'interpolated/{county}/2023/running_summary.csv'))
df_vmt = df_vmt[df_vmt['pollutantID']=="98"].groupby('veh_type').sum()[['intrazonal_vmt','interzonal_vmt','daily_vmt_total']]

# order rows

df_vmt = df_vmt.loc[veh_type_list]

# add total row
df_vmt.loc['Total'] = df_vmt[['intrazonal_vmt','interzonal_vmt','daily_vmt_total']].sum()

df_vmt.to_csv(r"Y:\Air Quality\King County Emissions Inventory\2023\king_county_vmt.csv")

df_vmt

,intrazonal_vmt,interzonal_vmt,daily_vmt_total
veh_type,,,
light,"440,918.9","39,696,766.8","40,137,685.7"
medium,"1,602.5","1,954,736.3","1,956,338.9"
heavy,"1,275.5","1,417,139.8","1,418,415.3"
transit,0.0,"128,670.9","128,670.9"
Total,"443,796.9","43,197,313.8","43,641,110.8"


# City Emissions and VMT

### VMT

In [6]:
city_vmt_results_df = pd.DataFrame()

dir_path = "Y:/Air Quality/King County Emissions Inventory/2023/data/city/interzonal_vmt"
city_list = sorted([f for f in os.listdir(dir_path)
                    if os.path.isfile(os.path.join(dir_path, f)) and f.lower().endswith('.csv')])

# derive city names (filenames without extension)
city_list = [os.path.splitext(f)[0] for f in city_list]

for city in city_list:
    # Interzonal VMT
    df = pd.read_csv(f"Y:/Air Quality/King County Emissions Inventory/2023/data/city/interzonal_vmt/{city}.csv")



    df['light'] = df['sov_vmt'] + df['hov2_vmt'] + df['hov3_vmt']
    df['medium'] = df['medium_truck_vmt']
    df['heavy'] = df['heavy_truck_vmt']

    df_city = pd.DataFrame(df[['light','medium','heavy']].sum(), columns=['interzonal_vmt'])
    df_city['city'] = city

    # Intrazonal VMT
    df = pd.read_csv(f"Y:/Air Quality/King County Emissions Inventory/2023/data/city/intrazonal_vmt/{city}.csv")

    df_city.loc['light','intrazonal_vmt'] = df[df['vehicle_type'].isin(['sov','hov2','hov3'])].sum()[['VMT']].values[0]
    df_city.loc['medium','intrazonal_vmt'] = df[df['vehicle_type'].isin(['mediumtruck'])].sum()[['VMT']].values[0]
    df_city.loc['heavy','intrazonal_vmt'] = df[df['vehicle_type'].isin(['heavytruck'])].sum()[['VMT']].values[0]

    city_vmt_results_df = pd.concat([city_vmt_results_df, df_city])

In [7]:
city_vmt_results_df['daily_vmt_total'] = city_vmt_results_df['interzonal_vmt'] + city_vmt_results_df['intrazonal_vmt']

In [8]:
city_vmt_results_df[['city','interzonal_vmt','intrazonal_vmt','daily_vmt_total']]

,city,interzonal_vmt,intrazonal_vmt,daily_vmt_total
light,Algona,"142,394.8",154.6,"142,549.3"
medium,Algona,"11,011.5",1.4,"11,012.9"
heavy,Algona,"11,083.9",1.0,"11,084.9"
light,Auburn,"1,663,283.3","6,119.3","1,669,402.6"
medium,Auburn,"82,291.8",50.9,"82,342.7"
...,...,...,...,...
medium,Woodinville,"13,008.4",5.2,"13,013.6"
heavy,Woodinville,"6,355.8",0.8,"6,356.6"
light,Yarrow Point,"24,821.4",9.3,"24,830.7"
medium,Yarrow Point,"2,070.9",0.0,"2,070.9"


In [9]:
city_vmt_results_df[['city','interzonal_vmt','intrazonal_vmt','daily_vmt_total']].to_csv(r"Y:\Air Quality\King County Emissions Inventory\2023\city_vmt.csv")

### Emissions

In [10]:
# Calculate emissions as a share based on VMT
# Load county emissions from above
county_emissions_results

,pollutant_name,start_tons,running_daily_tons,total_daily_tons
veh_type,,,,
light,CO2 Equivalent,"1,095.9","15,655.1","16,751.0"
medium,CO2 Equivalent,64.9,"1,431.4","1,496.4"
heavy,CO2 Equivalent,1.4,"2,720.3","2,721.7"
transit,CO2 Equivalent,0.6,249.4,250.0
light,Methane,0.7,0.5,1.2
medium,Methane,0.2,0.3,0.4
heavy,Methane,0.0,0.2,0.2
transit,Methane,0.0,0.3,0.3
light,N20,0.3,0.2,0.4


In [11]:

city_emissions_df = city_vmt_results_df.merge(county_emissions_results, left_index=True, right_index=True, how='left')
county_vmt = df_vmt.copy()
county_vmt.rename(columns={'intrazonal_vmt': 'county_intrazonal_vmt_total',
                           'interzonal_vmt': 'county_interzonal_vmt_total',
                            'daily_vmt_total': 'county_daily_vmt_total'
                           }, inplace=True)
city_emissions_df = city_emissions_df.merge(county_vmt, left_index=True, right_index=True, how='left')

In [12]:
city_emissions_df[f'daily_vmt_share'] = city_emissions_df[f'daily_vmt_total']/city_emissions_df[f'county_daily_vmt_total'] 

In [13]:

for col in ['start_tons', 'running_daily_tons','total_daily_tons']:
    city_emissions_df[f'city_{col}'] = city_emissions_df[col]*city_emissions_df['daily_vmt_share']

In [14]:
city_emissions_df[['city','city_start_tons', 'city_running_daily_tons','city_total_daily_tons']].to_csv(r"Y:\Air Quality\King County Emissions Inventory\2023\city_emissions.csv")

In [15]:
city_emissions_df[city_emissions_df['pollutant_name'] == 'CO2 Equivalent'][['city','city_start_tons', 'city_running_daily_tons','city_total_daily_tons']]

,city,city_start_tons,city_running_daily_tons,city_total_daily_tons
light,Algona,3.9,55.6,59.5
medium,Algona,0.4,8.1,8.4
heavy,Algona,0.0,21.3,21.3
light,Auburn,45.6,651.1,696.7
medium,Auburn,2.7,60.2,63.0
...,...,...,...,...
medium,Woodinville,0.4,9.5,10.0
heavy,Woodinville,0.0,12.2,12.2
light,Yarrow Point,0.7,9.7,10.4
medium,Yarrow Point,0.1,1.5,1.6


In [16]:
city_emissions_df[city_emissions_df['pollutant_name'] == 'CO2 Equivalent'].sum()

interzonal_vmt                                                      38,492,878.9
city                           AlgonaAlgonaAlgonaAuburnAuburnAuburnBeaux Arts...
intrazonal_vmt                                                          91,867.8
daily_vmt_total                                                     38,584,746.7
pollutant_name                 CO2 EquivalentCO2 EquivalentCO2 EquivalentCO2 ...
start_tons                                                              45,327.6
running_daily_tons                                                     772,465.3
total_daily_tons                                                       817,792.8
county_intrazonal_vmt_total                                         17,308,079.6
county_interzonal_vmt_total                                      1,679,677,073.7
county_daily_vmt_total                                           1,696,985,153.2
daily_vmt_share                                                              2.6
city_start_tons             

### Unincorporated Area Emissions
Calculate as remainder between city totals and county total

In [17]:
city_emissions_df[city_emissions_df['pollutant_name'] == 'CO2 Equivalent']

,interzonal_vmt,city,intrazonal_vmt,daily_vmt_total,pollutant_name,start_tons,running_daily_tons,total_daily_tons,county_intrazonal_vmt_total,county_interzonal_vmt_total,county_daily_vmt_total,daily_vmt_share,city_start_tons,city_running_daily_tons,city_total_daily_tons
light,"142,394.8",Algona,154.6,"142,549.3",CO2 Equivalent,"1,095.9","15,655.1","16,751.0","440,918.9","39,696,766.8","40,137,685.7",0.0,3.9,55.6,59.5
medium,"11,011.5",Algona,1.4,"11,012.9",CO2 Equivalent,64.9,"1,431.4","1,496.4","1,602.5","1,954,736.3","1,956,338.9",0.0,0.4,8.1,8.4
heavy,"11,083.9",Algona,1.0,"11,084.9",CO2 Equivalent,1.4,"2,720.3","2,721.7","1,275.5","1,417,139.8","1,418,415.3",0.0,0.0,21.3,21.3
light,"1,663,283.3",Auburn,"6,119.3","1,669,402.6",CO2 Equivalent,"1,095.9","15,655.1","16,751.0","440,918.9","39,696,766.8","40,137,685.7",0.0,45.6,651.1,696.7
medium,"82,291.8",Auburn,50.9,"82,342.7",CO2 Equivalent,64.9,"1,431.4","1,496.4","1,602.5","1,954,736.3","1,956,338.9",0.0,2.7,60.2,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
medium,"13,008.4",Woodinville,5.2,"13,013.6",CO2 Equivalent,64.9,"1,431.4","1,496.4","1,602.5","1,954,736.3","1,956,338.9",0.0,0.4,9.5,10.0
heavy,"6,355.8",Woodinville,0.8,"6,356.6",CO2 Equivalent,1.4,"2,720.3","2,721.7","1,275.5","1,417,139.8","1,418,415.3",0.0,0.0,12.2,12.2
light,"24,821.4",Yarrow Point,9.3,"24,830.7",CO2 Equivalent,"1,095.9","15,655.1","16,751.0","440,918.9","39,696,766.8","40,137,685.7",0.0,0.7,9.7,10.4
medium,"2,070.9",Yarrow Point,0.0,"2,070.9",CO2 Equivalent,64.9,"1,431.4","1,496.4","1,602.5","1,954,736.3","1,956,338.9",0.0,0.1,1.5,1.6
